### `Conv2DPytTest` code

In [1]:
import numpy as np

In [2]:
class Conv2DPytTest:
    """ Computes convolution given the input parameters """
    
    """ * The class implementation will be along the lines of torch.nn.Conv2D in order to 
          enable comparison of this NumPy only implementation and seamless testing
        * Can expect extensive refactoring of the existing code in the days to come
        * As part of refactoring, some code will be de-modularized
        * Old code will be retained at the end of the notebook for reference
    """
    """
        TODO:
        * Parameter error checking and assertion
        * Implementing other features and caveats offered by nn.torch.Conv2D 
          (e.g., `groups` flag to enable depthwise convolution, uniform sampling of kernel weights etc.)
        * Optimizing code
    """
    
    def __init__(
        self, 
        in_channels, 
        out_channels, 
        kernel_size, 
        padding = 0, 
        stride = 1, 
        dilation = 1, 
        groups = 1, 
        bias = True, 
        padding_mode = 'zeros', 
        device = None, 
        dtype = None, 
        verbose = True, 
        debug = False
        ):
        super(Conv2DPytTest, self).__init__()
        
        ''' mandatory parameters '''
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        
        ''' optional parameters '''
        self.padding = padding
        self.stride = stride
        self.dilation = dilation
        
        ''' optional parameters (dummy, yet to be implemented)'''
        self.groups = groups
        self.bias = bias
        self.padding_mode = padding_mode
        self.device = device
        self.dtype = dtype
        
        ''' additional parameters (different from torch.nn.Conv2D)'''
        self.verbose = verbose
        self.verboseprint = print if self.verbose else lambda *a, **k: None
        self.debug = debug
        self.debugprint = print if self.debug else lambda *a, **k: None
        self.print_params()
    
    def print_params(self):
        self.verboseprint('*** parameters ***')
        self.verboseprint('in_channels: {}, out_channels: {}, kernel_size: {}'.format(self.in_channels, self.out_channels, self.kernel_size))
        self.verboseprint('padding: {}, stride: {}, dilation factor: {}'.format(self.padding, self.stride, self.dilation))
        self.verboseprint('groups: {}, bias: {}, padding_mode: {}, device: {}, dtype: {}'.format(self.groups, self.bias, self.padding_mode, self.device, self.dtype))
        self.verboseprint('\n')
    
    def add_padding(self, _input):
        # add zero padding based on the input parameters
        if self.padding != 0:
            _input = np.array([[np.pad(channel,self.padding, 'constant', constant_values = 0) for channel in batch] for batch in _input])    
            self.verboseprint('*** padded input image ***')
            self.verboseprint('input batches: {}, input channels: {}, input height: {}, input weight: {}'.format(_input.shape[0], _input.shape[1], _input.shape[2], _input.shape[3]))
            self.verboseprint(_input)
            self.verboseprint('\n')
        return _input
    
    def create_kernels(self, kernels):
        # use the provided kernels or create random kernels based on the input kernel parameters
        if kernels is not None:
            self.verboseprint('*** kernels ***')
            self.verboseprint('kernels: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.out_channels, self.in_channels, self.kernel_size[0], self.kernel_size[1]))
            kernels = self.dilate_kernels(kernels)
            return kernels
        
        ''' Below code never executes as we are passing `kernels` as input - can be commented/removed '''
        kernels = []
        self.verboseprint('*** kernels ***')
        self.verboseprint('kernels: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.out_channels, self.in_channels, self.kernel_size[0], self.kernel_size[1]))
        for k in range(self.out_channels):
            kernel = np.random.rand(self.in_channels, self.kernel_size[0], self.kernel_size[1]) # define a random kernel based on the kernel parameters
            if self.debug:
                kernel = k * np.ones_like(kernel)
            kernels.append(kernel)
            self.verboseprint('kernel {}'.format(k))
            self.verboseprint(kernel)
        self.verboseprint('\n')
        kernels = self.dilate_kernels(kernels)
        return kernels 
    
    def dilate_kernels(self, kernels):
        # dilate a kernel
        dil_ker_h = self.dilation * (self.kernel_size[0] - 1) + 1
        dil_ker_w = self.dilation * (self.kernel_size[1] - 1) + 1
        dil_kernels = []
        for kernel in kernels:
            dil_kernel = []
            for channel in kernel:
                dil_channel = np.zeros((dil_ker_h, dil_ker_w))
                for row in range(len(channel)):
                    for col in range(len(channel[0])):
                        dil_channel[self.dilation*row][self.dilation*col] = channel[row][col]
                dil_kernel.append(dil_channel.tolist())
            dil_kernels.append(dil_kernel)
        kernels, self.kernel_size = dil_kernels, (dil_ker_h, dil_ker_w)
        self.verboseprint('*** dilated kernels ***')
        self.verboseprint('kernels: {}, dilation factor: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.out_channels, self.dilation, self.in_channels, self.kernel_size[0], self.kernel_size[1]))
        for k in range(self.out_channels):
            self.verboseprint('kernel {}'.format(k))
            self.verboseprint(kernels[k])
        self.verboseprint('\n')
        return kernels
    
    def compute_out_vol(self, _input):
        # compute output volume from the input and kernel parameters
        _input_n, _, _input_h, _input_w = _input.shape
        
        out_n = int(_input_n)
        out_c = int(self.out_channels)
        out_h = int((_input_h - self.kernel_size[0])/self.stride) + 1
        out_w = int((_input_w - self.kernel_size[1])/self.stride) + 1
        return out_n, out_c, out_h, out_w
    
    def forward(self, _input, kernels):
        # create output from the input and kernel parameters 
        _input = self.add_padding(_input)
        if self.debug:
            for b in range(_input.shape[0]):
                _input[b] = (b+1) * np.ones_like(_input[b]) # define an image of all ones (twos etc.) based on the input parameters
        kernels = self.create_kernels(kernels)
        out_n, out_c, out_h, out_w = self.compute_out_vol(_input)
        output = np.zeros([out_n, out_c, out_h, out_w])
        # parse through every element of the output and compute the convolution value for that element
        for b in range(out_n):
            for k in range(out_c):
                for h in range(out_h):
                    for w in range(out_w):
                        # output[b, k, h, w] += self.convolve(h, w, k, b, _input, kernels)
                        # convolve kernel over the input slices
                        self.debugprint('kernel indices, image indices')
                        self.debugprint('[c, h, w]', '[n, c, h, w]')
                        convol_sum = 0
                        ker_c = self.in_channels
                        ker_h = self.kernel_size[0]
                        ker_w = self.kernel_size[1]
                        for c_ker in range(ker_c):
                            for h_ker in range(ker_h):
                                for w_ker in range(ker_w):
                                    self.debugprint([c_ker, h_ker, w_ker], [b, c_ker, h_ker + self.stride*h, w_ker + self.stride*w])
                                    convol_sum += kernels[k][c_ker][h_ker][w_ker] * _input[b][c_ker][h_ker + self.stride*h][w_ker + self.stride*w]
                        self.debugprint('\n')
                        output[b, k, h, w] += convol_sum
        self.verboseprint('*** Conv2DPytTest output ***')
        output_shape = output.shape
        self.verboseprint('output batches: {}, ouput channels: {}, output height: {}, output weight: {}'.format(output_shape[0], output_shape[1], output_shape[2], output_shape[3]))
        assert((out_n, out_c, out_h, out_w) == output_shape)
        self.verboseprint(output)
        self.verboseprint('\n')
        return output

### Standalone test

In [3]:
debug = False # DO NOT CHANGE THIS while using Conv2DPytTest - TODO: modify this flag to `test`

in_channels = 2 # input channels
out_channels = 3 # output channels
kernel_size = (2, 2) # kernel size

padding = 1 # padding (optional)
stride = 2 # stride (optional)
dilation = 1 # dilation factor (optional)

in_batches = 2 # input batches
in_h = 4 # input height
in_w = 4 # input weight

_input = np.random.rand(in_batches, in_channels, in_h, in_w) # define a random image based on the input parameters
kernels = []
for k in range(out_channels):
    kernel = np.random.rand(in_channels, kernel_size[0], kernel_size[1]) # define a random kernel based on the kernel parameters
    kernels.append(kernel)

In [4]:
# get Conv2DPytTest output with the random inputs

conv2dpyttest = Conv2DPytTest(in_channels, out_channels, kernel_size, stride = stride, padding = padding, dilation = dilation, debug = debug) # call an instance of the class with the input parameters 
_output = conv2dpyttest.forward(_input, kernels) # perform convolution

*** parameters ***
in_channels: 2, out_channels: 3, kernel_size: (2, 2)
padding: 1, stride: 2, dilation factor: 1
groups: 1, bias: True, padding_mode: zeros, device: None, dtype: None


*** padded input image ***
input batches: 2, input channels: 2, input height: 6, input weight: 6
[[[[0.         0.         0.         0.         0.         0.        ]
   [0.         0.83672081 0.36264696 0.5983482  0.59536027 0.        ]
   [0.         0.98458614 0.05046472 0.02306613 0.06208951 0.        ]
   [0.         0.73242363 0.46557687 0.59200138 0.16819485 0.        ]
   [0.         0.73433603 0.34704122 0.51759645 0.57526481 0.        ]
   [0.         0.         0.         0.         0.         0.        ]]

  [[0.         0.         0.         0.         0.         0.        ]
   [0.         0.5092462  0.85067404 0.6605529  0.79828189 0.        ]
   [0.         0.32925848 0.5330689  0.80769057 0.27678479 0.        ]
   [0.         0.79653086 0.89060773 0.33116185 0.56860589 0.        ]
   [0

In [5]:
# get PyTorch output with the same random inputs as above

import torch

x = torch.DoubleTensor(_input)
weights = torch.stack([torch.DoubleTensor(kernel) for kernel in kernels])
output = torch.nn.functional.conv2d(x, weights, stride = stride, padding = padding, dilation = dilation)
print("*** PyTorch output ***")
print(output)

*** PyTorch output ***
tensor([[[[0.6998, 1.4021, 0.9081],
          [1.0698, 2.0827, 0.6953],
          [0.4273, 0.8397, 0.3351]],

         [[0.1927, 0.3068, 0.2064],
          [0.6745, 0.7812, 0.2159],
          [0.4293, 0.6094, 0.1002]],

         [[1.0712, 1.6446, 0.8726],
          [1.3497, 2.3956, 0.6760],
          [0.2705, 0.8422, 0.3343]]],


        [[[0.8660, 1.1160, 0.3432],
          [0.8873, 1.0791, 1.0411],
          [0.3299, 0.8491, 0.8500]],

         [[0.2572, 0.2889, 0.1733],
          [0.4370, 0.6371, 0.4732],
          [0.2664, 0.4957, 0.4734]],

         [[1.4366, 1.3444, 0.5357],
          [1.1532, 1.8467, 1.2066],
          [0.2476, 0.7481, 1.1022]]]], dtype=torch.float64)


In [6]:
# compare outputs of conv-Numpy and PyTorch
print(torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output))) # need to round the output due to precision difference

True


### Extensive tests

In [7]:
def valid_params(num_tests):
    # generates `num_samples` number of valid input and kernel parameters 
    params_list = []
    sample_count = 0
    while sample_count < num_tests:
        in_channels = np.random.randint(20) + 1 # input channels
        out_channels = np.random.randint(40) + 1 # output channels

        ker_h = np.random.randint(20) + 1
        ker_w = np.random.randint(20) + 1
        kernel_size = (ker_h, ker_w) # kernel size

        padding = np.random.randint(10) + 1 # padding (optional)
        stride = np.random.randint(5) + 1 # stride (optional)
        dilation = np.random.randint(10) + 1 # dilation factor (optional)

        in_batches = np.random.randint(5) + 1 # input batches
        in_h = np.random.randint(30) + 5 # input height
        in_w = np.random.randint(30) + 5 # input weight
    
        ker_h_flag, ker_w_flag, out_h_flag, out_w_flag = True, True, True, True
        
        if in_h + 2 * padding < dilation * (ker_h - 1) + 1: # check if (dilated) ker_h is valid
            ker_h_flag = False
        if in_w + 2 * padding < dilation * (ker_w - 1) + 1: # check if (dilated) ker_w is valid
            ker_w_flag = False
        if ((in_h + 2 * padding - (dilation * (ker_h - 1) + 1)) / stride) + 1 < 0: # check if out_h is valid
            out_h_flag = False
        if ((in_w + 2 * padding - (dilation * (ker_w - 1) + 1)) / stride) + 1 < 0: # check if out_w is valid
            out_w_flag = False
            
        if ker_h_flag and ker_w_flag and out_h_flag and out_w_flag:
            params_list.append({'in_channels': in_channels, 'out_channels': out_channels, 'kernel_size': kernel_size,
                          'padding': padding, 'stride': stride, 'dilation': dilation, 'in_batches': in_batches,
                          'in_h': in_h, 'in_w': in_w})
            sample_count += 1
    return params_list

In [8]:
# for loop sweeping different input parameters and testing the outputs of Conv2DPytTest and PyTorch
from tqdm import tqdm

num_tests = 50
num_passed = 0
params_list = valid_params(num_tests)
print('Number of tests: {}\n\n'.format(len(params_list)))

for i, params in enumerate(tqdm(params_list)):
    print('Test: {}\nParams: {}'.format(i, params))
    debug = False # DO NOT CHANGE THIS while using Conv2DPytTest - TODO: modify this flag to `test`

    in_channels = params['in_channels'] # input channels
    out_channels = params['out_channels'] # output channels
    
    kernel_size = params['kernel_size'] # kernel size

    padding = params['padding'] # padding (optional)
    stride = params['stride'] # stride (optional)
    dilation = params['dilation'] # dilation factor (optional)

    in_batches = params['in_batches'] # input batches
    in_h = params['in_h'] # input height
    in_w = params['in_w'] # input weight
    
    _input = np.random.rand(in_batches, in_channels, in_h, in_w) # define a random image based on the input parameters
    kernels = []
    for k in range(out_channels):
        kernel = np.random.rand(in_channels, kernel_size[0], kernel_size[1]) # define a random kernel based on the kernel parameters
        kernels.append(kernel)
    
    try:
        # get Conv2DPytTest output with the random inputs

        conv2dpyttest = Conv2DPytTest(in_channels, out_channels, kernel_size, stride = stride, padding = padding, dilation = dilation, debug = debug, verbose = False) # call an instance of the class with the input parameters 
        _output = conv2dpyttest.forward(_input, kernels) # perform convolution

        # get PyTorch output with the same random inputs as above

        x = torch.DoubleTensor(_input)
        weights = torch.stack([torch.DoubleTensor(kernel) for kernel in kernels])
        output = torch.nn.functional.conv2d(x, weights, stride = stride, padding = padding, dilation = dilation)
        
    except Exception as e:
        print(e)
        pass
    
    # compare outputs of conv-Numpy and PyTorch
    result = torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output)) # need to round the output due to precision difference
    print('Result: {}\n\n'.format(result))
    if result:
        num_passed += 1

print('{} out of {} ({}%) tests passed'.format(num_passed, num_tests, float(100 * num_passed / num_tests)))
    

Number of tests: 50




  0%|                                                    | 0/50 [00:00<?, ?it/s]

Test: 0
Params: {'in_channels': 18, 'out_channels': 33, 'kernel_size': (4, 9), 'padding': 8, 'stride': 3, 'dilation': 5, 'in_batches': 5, 'in_h': 6, 'in_w': 30}


  2%|▉                                           | 1/50 [00:33<26:58, 33.03s/it]

Result: True


Test: 1
Params: {'in_channels': 15, 'out_channels': 31, 'kernel_size': (5, 1), 'padding': 7, 'stride': 5, 'dilation': 5, 'in_batches': 4, 'in_h': 25, 'in_w': 27}


...

Result: True


Test: 47
Params: {'in_channels': 5, 'out_channels': 14, 'kernel_size': (11, 13), 'padding': 5, 'stride': 5, 'dilation': 2, 'in_batches': 4, 'in_h': 25, 'in_w': 18}


 96%|█████████████████████████████████████████▎ | 48/50 [14:47<00:14,  7.05s/it]

Result: True


Test: 48
Params: {'in_channels': 11, 'out_channels': 3, 'kernel_size': (14, 18), 'padding': 6, 'stride': 5, 'dilation': 1, 'in_batches': 4, 'in_h': 5, 'in_w': 9}
Result: True


Test: 49
Params: {'in_channels': 10, 'out_channels': 10, 'kernel_size': (6, 3), 'padding': 9, 'stride': 3, 'dilation': 1, 'in_batches': 2, 'in_h': 16, 'in_w': 10}


100%|███████████████████████████████████████████| 50/50 [14:48<00:00, 17.77s/it]

Result: True


50 out of 50 (100.0%) tests passed


### Modular code for `forward()`

### Old `Conv2D` code  for reference